In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import itertools
from scipy.spatial import cKDTree
import geopy.distance
from operator import itemgetter

In [2]:
def ckdnearest_hospital(brick_kilns, hospital_df):
    A = np.vstack([np.array(geom) for geom in brick_kilns[['lon', 'lat']].values])
    B = np.vstack([np.array(geom) for geom in hospital_df[['Longitude', 'Latitude']].values])
    ckd_tree = cKDTree(B)
    dist, hospital_idx = ckd_tree.query(A, k=1)
    closest_hospital_point = B[hospital_idx]
    gdf = pd.concat(
        [brick_kilns,
            pd.DataFrame(closest_hospital_point, columns=['closest_hospital_long', 'closest_hospital_lat']),
            pd.Series(dist, name='dist')], axis=1)
    return gdf

def cal_bk_hosp_dist(path, hospital_df):
    state_bk = pd.read_csv(path)
    bk_hospital_mapping = ckdnearest_hospital(state_bk, hospital_df)
    bk_hospital_mapping['distance_km'] = 0
    for i in range(len(bk_hospital_mapping)):
        bk_hospital_mapping['distance_km'][i] = geopy.distance.distance(
            (bk_hospital_mapping['lat'][i], bk_hospital_mapping['lon'][i]),
            (bk_hospital_mapping['closest_hospital_lat'][i], bk_hospital_mapping['closest_hospital_long'][i])
        ).km
    return bk_hospital_mapping

In [9]:
brick_kilns_paths = {
    'All' : '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/combined_file.csv',
    "Punjab": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/punjab.csv',
    "Haryana": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/haryana.csv',
    "Bihar": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/bihar.csv',
    "Uttar Pradesh": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/uttar_pradesh.csv',
    "West Bengal": '/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/west_bengal.csv',
}

state = 'West Bengal'
# Load brick kilns data for the selected state
brick_kilns_path = brick_kilns_paths[state]
brick_kilns = pd.read_csv(brick_kilns_path)

hospital_df = pd.read_csv('/home/shataxi.dubey/shataxi_work/Monitoring_Brick_Kiln_Compliance/India_Hospital_Data.csv')
hospital_df = hospital_df.rename(columns={'lon': 'Longitude', 'lat': 'Latitude'})

bk_hospital_mapping = cal_bk_hosp_dist(brick_kilns_path, hospital_df)
bk_hospital_mapping.to_csv(f'./{state}_bk_hospital_dist.csv')

/tmp/ipykernel_206226/3264543993.py:18: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  bk_hospital_mapping['distance_km'][i] = geopy.distance.distance(
/tmp/ipykernel_206226/3264543993.py:18: SettingWithCopyWarning: 
A value is trying to be s